In [78]:
import os
import pymongo
import pandas as pd

In [79]:
myclient = pymongo.MongoClient('mongodb://localhost:27017/')
dblist = myclient.list_database_names()

In [80]:
dblist

['admin', 'config', 'local', 'quantaxis']

In [81]:
db_quantaxis = myclient['quantaxis']

In [82]:
collections = db_quantaxis.list_collection_names()
print(collections)

['stock_xdxr', 'user_list', 'risk', 'etf_list', 'index_min', 'stock_info', 'portfolio', 'stock_block', 'stock_min', 'stock_adj', 'index_day', 'strategy', 'user', 'account', 'stock_list', 'future_day', 'usersetting', 'stock_day', 'index_list', 'future_list', 'future_min']


In [83]:
table = db_quantaxis['stock_day']

In [84]:
row = table.find_one()
row

{'_id': ObjectId('5e9165e96e17d64ee92905c7'),
 'open': 49.0,
 'close': 49.0,
 'high': 49.0,
 'low': 49.0,
 'vol': 32768.5,
 'amount': 5000.0,
 'date': '1991-04-03',
 'code': '000001',
 'date_stamp': 670608000.0}

In [114]:
# rows = table.find({'code': '000001'})
codes = [
    '601816', # 美凯龙
    '601816', # 中国高铁
    '601318', # 中国平安
    '600864', # 哈投股份
    '300708', # 聚灿光电
    '300347', # 泰格医药
    '300152', # 科融环境
    '002713', # 东易日盛
    '002466', # 天齐锂业
    '002351', # 漫步者
    '002049', # 紫光国微
    '000400', # 许继电气
    '000723', # 美锦能源
    '159977', # 创业板TH
    '159995', # 芯片ETF
]
codes = ['300027']
cursor = table.find({'code': {'$in': codes}})

In [86]:
cursor

In [115]:
res = pd.DataFrame([item for item in cursor])

In [116]:
res_new = res.assign(
    volume=res.vol,
    date=pd.to_datetime(res.date)
).drop_duplicates((['date', 'code'])).query('volume>1').set_index('date', drop=False)

In [117]:
final_res = res_new.loc[:, [
  'code',
  'open',
  'high',
  'low',
  'close',
  'volume',
  'amount',
  'date']
]

In [119]:
final_res.loc[[pd.to_datetime('2011-04-14'), pd.to_datetime('2011-04-15')]]

,code,open,high,low,close,volume,amount,date
date,,,,,,,,
2011-04-14,300027,27.10,27.30,26.75,26.9,26406.0,71263504.0,2011-04-14
2011-04-15,300027,14.79,14.83,14.51,14.6,32896.0,48017348.0,2011-04-15


In [141]:
stock_adj = db_quantaxis['stock_adj']

{'_id': ObjectId('5eaad0e07a181ec9c2a09e3e'),
 'date': '1991-04-03',
 'code': '000001',
 'adj': 0.0039214009}

In [145]:
cursor = stock_adj.find({'code': {'$in': codes}})

In [146]:
adj = pd.DataFrame([item for item in cursor])

In [179]:
adj.loc[(adj.date >= '2011-04-11') & (adj.date <= '2011-04-15')]

,_id,adj,code,date
344,5eaad4407a181ec9c2e4a717,0.133010,300027,2011-04-11
345,5eaad4407a181ec9c2e4a718,0.133010,300027,2011-04-12
346,5eaad4407a181ec9c2e4a719,0.133010,300027,2011-04-13
347,5eaad4407a181ec9c2e4a71a,0.133010,300027,2011-04-14
348,5eaad4407a181ec9c2e4a71b,0.241211,300027,2011-04-15


In [180]:
26.9 * 0.133010

3.5779689999999995

In [181]:
14.6 * 0.241211

3.5216806